In [1]:
# TRANSFER LEARNING FINE TUNING -  MASS CALC BENIGN MALIGN NORMAL CLASSIFIER USING MAMMOGRAM PATCHES IN CBIS-DDSM
import numpy as np
import tensorflow.keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Convolution2D, Conv2D, MaxPooling2D, Lambda, GlobalMaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Activation, AveragePooling2D, Concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
#from tensorflow.keras.utils import np_utils
%matplotlib inline
tensorflow.keras.backend.set_image_data_format('channels_last')
import tensorflow as tf
import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
from sklearn import metrics
from tensorflow.keras.layers import ReLU
import time


vid='v004'

In [2]:
# Create CNN

IMG_SIZE=331

from tensorflow.keras.applications.nasnet import NASNetLarge
base_model=NASNetLarge(weights='imagenet',include_top=False, input_shape=(IMG_SIZE,IMG_SIZE,3))

for layer in base_model.layers:
    layer.trainable=False

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(512)(x)
x=ReLU()(x)
x=Dropout(0.5)(x)
x=Dense(128)(x) 
x=ReLU()(x)
preds=Dense(5,activation='softmax')(x)

model=Model(inputs=base_model.input,outputs=preds)

In [4]:
len(model.layers)

1046

In [5]:
from tensorflow.keras import optimizers
model.compile(optimizer=optimizers.Adam(learning_rate=1e-3), loss='categorical_crossentropy', metrics=['accuracy']) 

In [6]:
# Image preprocessing and data augmentation

batch_size=6

train_datagen = ImageDataGenerator(rescale=1./255,
                         horizontal_flip = True,
                         vertical_flip = True,
                         fill_mode="wrap",
                         width_shift_range = 0.2,
                         height_shift_range = 0.2,
                         zoom_range = 0.2,
                         rotation_range = 90,
                         shear_range=0.2
                        )

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator=train_datagen.flow_from_directory('CBIS_DDSM_PATCHES_01/train', 
                                                 target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='categorical',
                                                 shuffle=True)
valid_generator = valid_datagen.flow_from_directory('CBIS_DDSM_PATCHES_01/valid', 
                                                 target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='categorical',
                                                 shuffle=True)


Found 2170 images belonging to 5 classes.
Found 545 images belonging to 5 classes.


In [7]:
#Callbacks
#checkpoint_path="resnet50_train4/cp.ckpt"
def get_callbacks(name_weights, patience_lr):
    mcp_save = ModelCheckpoint(name_weights, save_best_only=True, monitor='val_loss', mode='min')
    reduce_lr_loss = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=patience_lr, verbose=2, min_delta=1e-4, mode='min')
    early_stop_cr=EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=10, verbose=2, mode='auto', restore_best_weights=True)
    return [mcp_save, reduce_lr_loss, early_stop_cr]


In [8]:
#Training stage 1
step_size_train=train_generator.n//train_generator.batch_size
step_size_valid=valid_generator.n//valid_generator.batch_size

name_weights = "mas_cal_ben_mal_class_nasnet50_weights_"+vid+".h5"
callbacks = get_callbacks(name_weights = name_weights, patience_lr=10)

model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   validation_steps=step_size_valid,
                   validation_data=valid_generator,
                   epochs=3,
                   verbose=1,
                   callbacks = callbacks)  

C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/3
361/361 [==============================] - 191s 409ms/step - loss: 1.4350 - accuracy: 0.3914 - val_loss: 1.3293 - val_accuracy: 0.4333


C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/3
361/361 [==============================] - 111s 308ms/step - loss: 1.2753 - accuracy: 0.4469 - val_loss: 1.1934 - val_accuracy: 0.4741
Epoch 3/3
361/361 [==============================] - 112s 310ms/step - loss: 1.2082 - accuracy: 0.4778 - val_loss: 1.1908 - val_accuracy: 0.4722


In [9]:
#Training stage 2
for layer in model.layers[:700]:
    layer.trainable=False
for layer in model.layers[700:]:  #last ~33% layers are trainable
    layer.trainable=True
    
model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

callbacks = get_callbacks(name_weights = name_weights, patience_lr=10)

model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   validation_steps=step_size_valid,
                   validation_data=valid_generator,
                   epochs=10,
                   verbose=1,
                   callbacks = callbacks)

Epoch 1/10
361/361 [==============================] - 175s 373ms/step - loss: 1.0683 - accuracy: 0.5573 - val_loss: 0.9585 - val_accuracy: 0.6037
Epoch 2/10
361/361 [==============================] - 139s 384ms/step - loss: 0.8370 - accuracy: 0.6456 - val_loss: 1.1563 - val_accuracy: 0.5833
Epoch 3/10
361/361 [==============================] - 138s 383ms/step - loss: 0.7378 - accuracy: 0.6811 - val_loss: 0.9955 - val_accuracy: 0.6704
Epoch 4/10
361/361 [==============================] - 138s 381ms/step - loss: 0.6963 - accuracy: 0.7075 - val_loss: 0.8800 - val_accuracy: 0.6815
Epoch 5/10
361/361 [==============================] - 139s 385ms/step - loss: 0.6709 - accuracy: 0.7190 - val_loss: 0.6584 - val_accuracy: 0.7463
Epoch 6/10
361/361 [==============================] - 138s 383ms/step - loss: 0.6055 - accuracy: 0.7384 - val_loss: 1.1753 - val_accuracy: 0.6111
Epoch 7/10
361/361 [==============================] - 130s 359ms/step - loss: 0.5790 - accuracy: 0.7639 - val_loss: 1.1710 -

In [10]:
#Training stage 3
#time.sleep(360)
for layer in model.layers:  #All layers are trainable
    layer.trainable=True
    
model.compile(optimizer=optimizers.Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
callbacks = get_callbacks(name_weights = name_weights, patience_lr=5)

model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   validation_steps=step_size_valid,
                   validation_data=valid_generator,
                   epochs=150,
                   verbose=1,
                   callbacks = callbacks)

Epoch 1/150
361/361 [==============================] - 389s 919ms/step - loss: 0.8093 - accuracy: 0.6705 - val_loss: 1.0240 - val_accuracy: 0.6815
Epoch 2/150
361/361 [==============================] - 328s 909ms/step - loss: 0.6201 - accuracy: 0.7542 - val_loss: 0.9033 - val_accuracy: 0.7093
Epoch 3/150
361/361 [==============================] - 328s 908ms/step - loss: 0.5900 - accuracy: 0.7666 - val_loss: 0.8573 - val_accuracy: 0.7148
Epoch 4/150
361/361 [==============================] - 329s 910ms/step - loss: 0.5380 - accuracy: 0.7782 - val_loss: 0.8237 - val_accuracy: 0.7259
Epoch 5/150
361/361 [==============================] - 329s 911ms/step - loss: 0.5051 - accuracy: 0.7819 - val_loss: 0.7945 - val_accuracy: 0.7389
Epoch 6/150
361/361 [==============================] - 329s 910ms/step - loss: 0.4841 - accuracy: 0.7879 - val_loss: 0.8014 - val_accuracy: 0.7278
Epoch 7/150
361/361 [==============================] - 330s 913ms/step - loss: 0.4893 - accuracy: 0.8004 - val_loss: 0

In [11]:
model.metrics_names

['loss', 'accuracy']

In [12]:
# Validation accuracy, validation data confusion matrix, area under the roc score
valid_generator = valid_datagen.flow_from_directory('CBIS_DDSM_PATCHES_01/valid', 
                                                 target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='categorical',
                                                 shuffle=False)

# Ref: https://gist.github.com/RyanAkilos/3808c17f79e77c4117de35aa68447045 accessed on 5 Feb 2020
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score, roc_auc_score, f1_score, matthews_corrcoef


Y_pred = model.predict_generator(valid_generator)
y_pred = np.argmax(Y_pred, axis=1)
ras=roc_auc_score(valid_generator.classes, Y_pred,multi_class='ovr')


print('Confusion Matrix:')
print(confusion_matrix(valid_generator.classes, y_pred))
print('Accuracy:', accuracy_score(valid_generator.classes, y_pred))
print('ROC AUC score:', ras)



Found 545 images belonging to 5 classes.


C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\engine\training.py:2001: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


Confusion Matrix:
[[ 79  22   2   4   2]
 [ 22  62   4  20   1]
 [  2   0  58  46   3]
 [  0   0  10  99   0]
 [  2   0   0   3 104]]
Accuracy: 0.7376146788990826
ROC AUC score: 0.9357335241141318


In [13]:
train_generator.filenames

['calc_ben\\calc_ben_0001.png',
 'calc_ben\\calc_ben_0002.png',
 'calc_ben\\calc_ben_0003.png',
 'calc_ben\\calc_ben_0004.png',
 'calc_ben\\calc_ben_0006.png',
 'calc_ben\\calc_ben_0007.png',
 'calc_ben\\calc_ben_0008.png',
 'calc_ben\\calc_ben_0009.png',
 'calc_ben\\calc_ben_0017.png',
 'calc_ben\\calc_ben_0019.png',
 'calc_ben\\calc_ben_0020.png',
 'calc_ben\\calc_ben_0022.png',
 'calc_ben\\calc_ben_0023.png',
 'calc_ben\\calc_ben_0024.png',
 'calc_ben\\calc_ben_0025.png',
 'calc_ben\\calc_ben_0034.png',
 'calc_ben\\calc_ben_0035.png',
 'calc_ben\\calc_ben_0036.png',
 'calc_ben\\calc_ben_0038.png',
 'calc_ben\\calc_ben_0041.png',
 'calc_ben\\calc_ben_0049.png',
 'calc_ben\\calc_ben_0050.png',
 'calc_ben\\calc_ben_0051.png',
 'calc_ben\\calc_ben_0053.png',
 'calc_ben\\calc_ben_0054.png',
 'calc_ben\\calc_ben_0056.png',
 'calc_ben\\calc_ben_0057.png',
 'calc_ben\\calc_ben_0065.png',
 'calc_ben\\calc_ben_0067.png',
 'calc_ben\\calc_ben_0069.png',
 'calc_ben\\calc_ben_0071.png',
 'calc_b

In [14]:
valid_generator.filenames

['calc_ben\\calc_ben_0005.png',
 'calc_ben\\calc_ben_0018.png',
 'calc_ben\\calc_ben_0021.png',
 'calc_ben\\calc_ben_0033.png',
 'calc_ben\\calc_ben_0037.png',
 'calc_ben\\calc_ben_0039.png',
 'calc_ben\\calc_ben_0040.png',
 'calc_ben\\calc_ben_0052.png',
 'calc_ben\\calc_ben_0055.png',
 'calc_ben\\calc_ben_0066.png',
 'calc_ben\\calc_ben_0068.png',
 'calc_ben\\calc_ben_0070.png',
 'calc_ben\\calc_ben_0072.png',
 'calc_ben\\calc_ben_0081.png',
 'calc_ben\\calc_ben_0082.png',
 'calc_ben\\calc_ben_0089.png',
 'calc_ben\\calc_ben_0098.png',
 'calc_ben\\calc_ben_0102.png',
 'calc_ben\\calc_ben_0114.png',
 'calc_ben\\calc_ben_0115.png',
 'calc_ben\\calc_ben_0129.png',
 'calc_ben\\calc_ben_0137.png',
 'calc_ben\\calc_ben_0146.png',
 'calc_ben\\calc_ben_0147.png',
 'calc_ben\\calc_ben_0149.png',
 'calc_ben\\calc_ben_0150.png',
 'calc_ben\\calc_ben_0152.png',
 'calc_ben\\calc_ben_0166.png',
 'calc_ben\\calc_ben_0167.png',
 'calc_ben\\calc_ben_0184.png',
 'calc_ben\\calc_ben_0193.png',
 'calc_b

In [15]:
#Print estimation result for ensemble model
print(Y_pred)
print(valid_generator.classes)

[[9.97590542e-01 2.22828495e-03 1.49837477e-04 3.07033879e-05
  7.35449987e-07]
 [8.77884269e-01 5.94667085e-02 3.20832729e-02 3.01093124e-02
  4.56453272e-04]
 [2.64743179e-01 3.28247070e-01 1.81319177e-01 1.79533288e-01
  4.61573042e-02]
 ...
 [9.35956894e-04 8.53403355e-04 2.37501482e-03 1.64068758e-03
  9.94194925e-01]
 [1.34239078e-03 1.35212590e-03 1.40605755e-02 9.30279773e-03
  9.73942101e-01]
 [1.40177761e-03 1.54305797e-03 1.07569387e-03 7.72678060e-04
  9.95206773e-01]]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2